In [536]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [537]:
import yaml

In [538]:
from IPython.html.widgets import FloatProgress
from IPython.display import display

In [539]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [540]:
saut="\n"

### Préparation des matrices de traits

In [541]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [542]:
phonologicalMap="-N"
sampleNumber="03%s"%phonologicalMap
sampleType="BRASS"
sampleType="SILVER"
#samplePrefix="MGC-160104-%s-ext3-derivations-%s"%(sampleNumber,sampleType)
samplePrefix="MGC-160104-%s-ext3-160215-derivations-%s"%(sampleNumber,sampleType)
sampleFile=samplePrefix+".csv"
#analysisPrefix="MGC-160104-%s-ext3-derivations-%s"%(sampleNumber,sampleType)
analysisPrefix=samplePrefix

In [543]:
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [544]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [545]:
with open("2015-Data/"+analysisPrefix+'-TousDerives.pkl', 'rb') as input:
    tousDerives=pickle.load(input)

In [546]:
with open("2015-Data/MGC-160104-DerivesComplets.pkl", 'rb') as input:
    derivesComplets=pickle.load(input)
derivesComplets["FS"]=derivesComplets.apply(lambda x: recoder(x["FS"]),axis=1)
derivesComplets["FP"]=derivesComplets.apply(lambda x: recoder(x["FP"]),axis=1)

# Le tableau est assemblé à la main à partir des échantillons
- faire une boucle pour charger tous les échantillons

In [547]:
tableauDerives=(tousDerives[["lexeme","FS_x","FP_x","FS_y","FP_y"]]).rename(columns={"lexeme":"base"})
#tableauDerives
tableauEvaluation=pd.merge(tableauDerives,derivesComplets,on="base")
tableauEvaluation.loc[tableauEvaluation["FS"].isnull(),"FS"]=tableauEvaluation.loc[tableauEvaluation["FS"].isnull(),"FP"]
tableauEvaluation.loc[tableauEvaluation["FP"].isnull(),"FP"]=tableauEvaluation.loc[tableauEvaluation["FP"].isnull(),"FS"]

In [548]:
numerosSamples=["1","2","3"]
typesSamples=["B","S"]
nomsSamples=[n+t for (n,t) in (list(it.product(numerosSamples, typesSamples)))]
casesSamples=["FS","FP"]
samplesColumnsFormes=[c+"-"+n for (n,c) in (list(it.product(nomsSamples,casesSamples)))]
samplesColumns=["base","FS","FP"]+samplesColumnsFormes
samplesEmptyColumn=["" for element in samplesColumns]

In [549]:
def fNoLex(df):
    return ((df["FS_x"].isnull())&(df["FP_x"].isnull()))
def fFS(df):
    return ((df["FS_y"].notnull())&(df["FP_y"].isnull()))
def fFP(df):
    return ((df["FP_y"].notnull())&(df["FS_y"].isnull()))
def fFSFP(df):
    return ((df["FS_y"].notnull())&(df["FP_y"].notnull()))

def filtrePrecisionRappel(df,case,nom):
    return ((df[case].notnull())&(df["%s_x"%(case)].isnull())&(df["%s-%s"%(case,nom)].notnull()))

In [550]:
print (sampleNumber, sampleType)
for case in casesSamples:
    print (case,end="\t")
    ok=tableauEvaluation[(tableauEvaluation["%s_x"%case].isnull())&(tableauEvaluation["%s_y"%case].notnull())].apply(lambda x: x[case]==x["%s_y"%case],axis=1).sum()
    mauvais=tableauEvaluation[(tableauEvaluation["%s_x"%case].isnull())&(tableauEvaluation["%s_y"%case].notnull())].apply(lambda x: x[case]!=x["%s_y"%case],axis=1).sum()
    total=tableauEvaluation[tableauEvaluation["%s_x"%case].isnull()].apply(lambda x: x[case]==x[case],axis=1).sum()
    print (total, ok, mauvais)

03-N SILVER
FS	91 40 50
FP	91 40 50


In [551]:
print (sampleNumber, sampleType)
ok=tableauEvaluation[fNoLex(tableauEvaluation)&fFS(tableauEvaluation)].apply(lambda x: x["FS"]==x["FS_y"],axis=1).sum()
ok+=tableauEvaluation[fNoLex(tableauEvaluation)&fFP(tableauEvaluation)].apply(lambda x: x["FP"]==x["FP_y"],axis=1).sum()
ok+=tableauEvaluation[fNoLex(tableauEvaluation)&fFSFP(tableauEvaluation)].apply(lambda x: (x["FS"]==x["FS_y"]) and (x["FP"]==x["FP_y"]),axis=1).sum()
mauvais=tableauEvaluation[fNoLex(tableauEvaluation)&fFS(tableauEvaluation)].apply(lambda x: x["FS"]!=x["FS_y"],axis=1).sum()
mauvais+=tableauEvaluation[fNoLex(tableauEvaluation)&fFP(tableauEvaluation)].apply(lambda x: x["FP"]!=x["FP_y"],axis=1).sum()
mauvais+=tableauEvaluation[fNoLex(tableauEvaluation)&fFSFP(tableauEvaluation)].apply(lambda x: (x["FS"]!=x["FS_y"]) or (x["FP"]!=x["FP_y"]),axis=1).sum()
total=tableauEvaluation[fNoLex(tableauEvaluation)].apply(lambda x: x[case]==x[case],axis=1).sum()
#print (total, ok, mauvais)
print (total, ok["base"],mauvais["base"])


03-N SILVER
90 40.0 50.0


In [552]:
#tableauEvaluation[fNoLex(tableauEvaluation)&fFSFP(tableauEvaluation)].apply(lambda x: (x["FS"]!=x["FS_y"]) or (x["FP"]!=x["FP_y"]),axis=1)

In [553]:
samplesSelections=derivesComplets